In [1]:
import pandas as pd, numpy as np
%matplotlib notebook

In [2]:
values = pd.Series(['apple', 'Orange', 'apple', 'apple'] * 2)
values

0     apple
1    Orange
2     apple
3     apple
4     apple
5    Orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'Orange'], dtype=object)

In [4]:
values.unique()

array(['apple', 'Orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
Orange    2
dtype: int64

In [6]:
values.value_counts()

apple     6
Orange    2
dtype: int64

In [7]:
values = pd.Series([0,1,0,0] * 2)

In [8]:
dim = pd.Series(['apple', 'orange', 'banana'])

In [9]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [12]:
dim.take(values) # values should be numeric

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

# Categorical Type in pandas

In [13]:
fruits = ['apple', 'orange', 'apple', 'apple'] *2

In [14]:
N = len(fruits)

In [15]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)
                  },
                 columns=['basket_id', 'fruit', 'count', 'weight']
                 )

In [16]:
df

,basket_id,fruit,count,weight
0,0,apple,9,1.374664
1,1,orange,4,2.661723
2,2,apple,12,3.721504
3,3,apple,9,1.188651
4,4,apple,4,3.117899
5,5,orange,14,1.318892
6,6,apple,8,1.088668
7,7,apple,12,1.179743


In [17]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [18]:
c = fruit_cat.values

In [19]:
type(c)

pandas.core.arrays.categorical.Categorical

In [20]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [21]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [22]:
df['fruit'] = df['fruit'].astype('category')

In [23]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [24]:
df

,basket_id,fruit,count,weight
0,0,apple,9,1.374664
1,1,orange,4,2.661723
2,2,apple,12,3.721504
3,3,apple,9,1.188651
4,4,apple,4,3.117899
5,5,orange,14,1.318892
6,6,apple,8,1.088668
7,7,apple,12,1.179743


In [25]:
#Can also create pandas.Categorcal directly from other types of python sequences:

In [26]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [29]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2,  0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [30]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cats

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [31]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

# Computations with Categoricals

In [44]:
np.random.seed(12345)

In [45]:
draws = np.random.randn(1000)

In [46]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [51]:
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [52]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [53]:
pd.unique(bins.codes)

array([1, 2, 3, 0], dtype=int8)

In [54]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [55]:
bins = pd.Series(bins, name='quartile')

In [56]:
bins

0      Q2
1      Q3
2      Q2
3      Q2
4      Q4
       ..
995    Q3
996    Q2
997    Q1
998    Q3
999    Q4
Name: quartile, Length: 1000, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [57]:
results = pd.Series(draws).groupby(bins).agg(['count', 'min', 'max'])
results

,count,min,max
quartile,,,
Q1,250,-2.949343,-0.685484
Q2,250,-0.683066,-0.010115
Q3,250,-0.010032,0.628894
Q4,250,0.634238,3.927528


In [58]:
results.reset_index()['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

# Better performance with categoricals

In [59]:
N = 10000000

In [60]:
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N //4))
categories = labels.astype('category')


labels uses significantly more memory than categories:

In [54]:
labels.memory_usage()

80000128

In [55]:
categories.memory_usage()

10000320

In [56]:
%time _ = labels.astype('category') # The converion to category is not free, of course, but it is one time cost.

Wall time: 434 ms


GroupBy operations can be significantly faster with categoricals because the underlying algorithms use the integer-based codes array instead of an array of Strings.

# Categorical Methods

In [58]:
s = pd.Series(['a', 'b', 'c', 'd'] *2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [59]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [60]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [63]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [65]:

cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [66]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [70]:

cat_s[cat_s.isin(['a', 'b'])]

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [71]:
cat_s.cat.remove_unused_categories()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

# Other Methods
add_categories
as_ordered
as_unordered
remove_categories
remove_unused_categories
rename_categories
reorder_categories
set_categories

# Creating dummy variables for modeling AKA one-hot encoding

In [72]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [74]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1
